# Light Curves Simulation for all SNe types

In [1]:
import snsim
import importlib
import matplotlib.pyplot as plt
import numpy as np

#secutiry check for numpy parallelization when using dask
def limit_numpy(nthreads=1):
    """ """
    import os
    threads = str(nthreads)
    print(f"threads {threads}")
    os.environ["NUMEXPR_NUM_THREADS"] = threads
    os.environ["OMP_NUM_THREADS"] = threads
    os.environ["OPENBLAS_NUM_THREADS"] = threads
    os.environ["MKL_NUM_THREADS"] = threads
    os.environ["VECLIB_MAXIMUM_THREADS"] = threads
limit_numpy(1)

import dask
import pandas as pd

threads 1


### we use dask to speed up the simulation

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 15.32 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39751,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 15.32 GiB
Comm: tcp://127.0.0.1:36639,Total threads: 4
Dashboard: http://127.0.0.1:40519/status,Memory: 3.83 GiB
Nanny: tcp://127.0.0.1:38913,


### init the survey

In [3]:
survey_conf = {
        'survey_file': './survey_file_example.csv',
        'sig_psf': 0.0,
        'sig_zp': 0.01,
        'zp': 25,
        'gain': 1,
        'ra_size': 7.295,
        'dec_size': 7.465,
        'noise_key': ['skynoise', 'skysigADU'], 
         }

### init the generator obejct for all SNe

In [4]:
snia_gen = {
    'M0': 'jla',
    'sigM': 0.1,
    'sct_model': 'G10',
    'force_n': 1000 ,
    'model_name': 'salt2',
    'alpha': 0.14,
    'beta': 2.9,
    'dist_x1': 'N21',
    'dist_c': [-0.055, 0.023, 0.150]}

# for simplicity we use same configuration for SN core collapse
sncc_gen = {'M0': -19,
            'sigM': [1.5,1.5],
            'rate': 'ztf20',
            'model_name': 'vin19_nocorr'
            }

cosmology = {'name':'planck18'}



cmb = {'v_cmb': 0,
       'l_cmb':0,
       'b_cmb':0}

mw_dust = {'model': 'CCM89',}

#distribution of peculiar velocities of SNe
vpec_dist = {'mean_vpec':0,
             'sig_vpec':300}


## configuration dictionary, input of simulation

In [5]:
config_dic = {'data': {'write_path': './',
                       'sim_name': 'Test_simulation_allSN',
                       'write_format': 'parquet'},
              'survey_config': survey_conf,
              'sim_par': {'randseed': 1234, # Optional
                          'z_range': [0.01, 0.3]},
              'snia_gen': snia_gen,
              'sniipl_gen': sncc_gen,
              'sniin_gen': sncc_gen,
              'sniib_gen': sncc_gen,
              'snib_gen': sncc_gen,
              'snic_gen': sncc_gen,
              'snic-bl_gen': sncc_gen,
              'cosmology': cosmology,
              'mw_dust': mw_dust,
              'vpec_dist': vpec_dist,
              'cmb': cmb,
              'dask': {'use': True,
                       'nworkers': 4}#parameters to use dask, nworkers depends on your machine
             }

In [6]:
#config dask
cluster.scale(config_dic['dask']['nworkers'])

## Run the simulation

In [7]:
%%time
#initialize simulator object
#first time a bit long sncosmo has to download all the Templates
sim = snsim.Simulator(config_dic)

CPU times: user 9.56 s, sys: 346 ms, total: 9.9 s
Wall time: 9.43 s


In [8]:
# -- Generate n base param
param_tmp = sim.generators[0].gen_basic_par(1000, 1234, 
                                            min_max_t=True)

In [9]:
epochs, params = sim.survey.get_observations(
            param_tmp,
            phase_cut=None,
            nep_cut=sim.nep_cut,
            IDmin=0,
            use_dask=sim.config['dask']['use'],
            npartitions=sim.config['dask']['nworkers'])

In [ ]:
#run the simulation
sim.simulate()

      _______..__   __.         _______. __  .___  ___. 
     /       ||  \ |  |        /       ||  | |   \/   | 
    |   (----`|   \|  |       |   (----`|  | |  \  /  | 
     \   \    |  . `  |        \   \    |  | |  |\/|  | 
 .----)   |   |  |\   |    .----)   |   |  | |  |  |  | 
 |_______/    |__| \__|    |_______/    |__| |__|  |__| 
================================= Version : 0.4.5+dev ====== 
-----------------------------------------------------------

SIM NAME : Test_simulation_allSN
CONFIG FILE : No config file
SIM WRITE DIRECTORY : ./
SIMULATION RANDSEED : 1234

-----------------------------------------------------------

SURVEY FILE : ./survey_file_example.csv

First day in survey_file : 58000.00 MJD / 2017-09-04 00:00:00.000
Last day in survey_file : 58100.00 MJD / 2017-12-13 00:00:00.000

Survey effective duration is 100.00 days

Survey envelope area is 57.74 squared degrees (0.1 % of the sky)

No cut on survey file.

------------------------------------------------------

### After the simulations

In [ ]:
#the sim object has the attribute samples, which contain the data of all the object we simulated
sim.samples
#one sim.sample object for each generator 

In [ ]:
sim.samples[0].header
#header with useful parameter of each simulation

In [ ]:
#meta attribute return the parameters of each object, same as sim_lcs.attrs
sim.samples[0].meta[0]

In [ ]:
#return the simulated lightcurves
sim.samples[0].sim_lcs